# Import KNMI data

Reads in KNMI weather data and stores it in data/src/knmi.csv .

Run this file before running generate_ice_cream_sales.ipynb



https://www.daggegevens.knmi.nl/klimatologie/daggegevens?start=20220101&end=20241231&vars=TG:TN:TX:SQ:DR:RH:NG&stns[260]=1&fmt=csv

Variables:

- **TG**: Etmaalgemiddelde temperatuur
- **TN**: TN: Minimum temperatuur (in 0.1 graden Celsius)
- **TX**: Maximum temperatuur (in 0.1 graden Celsius)
- **SQ**: Zonneschijnduur (in 0.1 uur) berekend uit de globale straling (-1 voor <0.05 uur)
- **DR**: Duur van de neerslag (in 0.1 uur)
- **RH**: Etmaalsom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm)
- **NG**: Etmaalgemiddelde bewolking (bedekkingsgraad van de bovenlucht in achtsten, 9=bovenlucht onzichtbaar)

Stations:

- **260**: de Bilt. Other stations do not always have all data available.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request # For reading from URLs
import urllib.error # For reading from URLs
import io # For reading CSV data from a string.
import datetime # For generating timestamps


## Import the KNMI source data

First, import the raw KNMI source data.

In [ ]:
# Construct the csv url and import the raw data.

# Settings. Change as needed. See top of notebook for information on fields.
start = '20220101'
end = '20241231'
stations = [260]
fields = ['TG',
          'TN',
          'TX',
          'SQ',
          'DR',
          'RH',
          'NG'
          ]
base_url = 'https://www.daggegevens.knmi.nl/klimatologie/daggegevens'

num_lines_to_skip = 14 # KNMI adds information to the top of the file. Trial and error has shown that there are 14 lines of this stuff.

url = base_url + '?' + 'start=' + start + '&end=' + end

url += '&stns=' + ':'.join([str(station) for station in stations])

# stnlist = ['stsns[' + str(stsn) + ']=1' for stsn in stations]
# url += '&' + '&'.join(stnlist)

url += '&vars=' + ':'.join(fields)

url += '&fmt=csv'

print(url)

# Simply calling pd.read_csv on the url does not work, we have to
# read in the CSV data manually from the URL and then parse it.

csv_source = ''
try:
   with urllib.request.urlopen(url) as f:
      csv_source = f.read().decode('utf-8')
    #   print(f.read().decode('utf-8'))
except urllib.error.URLError as e:
    print(e.reason)

# Turn the source data into a proper CSV file and store it (for logging reasons).
df_raw = pd.read_csv(io.StringIO(csv_source), skiprows=num_lines_to_skip, on_bad_lines='warn')
df_raw.to_csv('data/src/knmi-import-' + 
              '-'.join(str(station) for station in stations) + 
              '_' + start + '-' + end + 
              datetime.datetime.now().strftime('%Y%m%d.%H.%M.%S'),
              index=False)


## Process the KNMI source data

Turn the KNMI source data into something we can use. Map column names to names that are more readable and do some basic feature engineering.

### Rename columns



In [ ]:
# What should fields be renamed to?
# The header row reads: # STN,YYYYMMDD,   TG,   TN,   TX,   SQ,   DR,   RH,   NG
translation_map = {
    '# STN' : 'station',
    'YYYYMMDD' : 'date',
    '   TG' : 'temp_avg',
    '   TN' : 'temp_min',
    '   TX' : 'temp_max',
    '   SQ' : 'sun_hr',
    '   DR' : 'rain_hr',
    '   RH' : 'rain_total_mm',
    '   NG' : 'cloud_cover_perc'
}
df_clean = df_raw.rename(columns=translation_map).copy()
df_clean

### Change units

Some columns have units that require explanation. Better to use units that make sense - and that match what the new headers say.

In [ ]:
# Temperatures are in 0.1 C. Turn them into C.
for temp_col in ['temp_avg', 'temp_min', 'temp_max']:
    df_clean[temp_col] = df_clean[temp_col] / 10

# Sun hours and rain hours are in 0.1 hour. Turn them into hours.
for hr_col in ['sun_hr', 'rain_hr']:
    df_clean[hr_col] = df_clean[hr_col] / 10

# Rain total is in 0.1 mm. Turn it into mm.
for mm_col in ['rain_total_mm']:
    df_clean[mm_col] = df_clean[mm_col] / 10

# Cloud cover is in 1/8th, with 9 being "cover complete".
# Let's decide that both 8 and 9 are 100
df_clean['cloud_cover_perc'] = df_clean['cloud_cover_perc'].map(lambda x : 1 if x >= 8  else x / 8)




# Fix values

Some columns have odd values that are used to signal special circumstances. Since this course is not about data cleaning, turn those values into values that are more meaningful.

In [ ]:
# -1 means: less than 0.05 hr. Just make it 0
df_clean['sun_hr'] = df_clean['sun_hr'].map(lambda x : 0 if x < 0 else x)

# -1 means: less than 0.05 mm. Just make it 0
df_clean['rain_total_mm'] = df_clean['rain_total_mm'].map(lambda x : 0 if x < 0 else x)

# Dates are in YYYYMMDD format. Turn them into something Pandas recognizes as dates.
df_clean['date'] = pd.to_datetime(df_clean['date'], format='%Y%m%d')

df_clean



## Drop the station column and save the file

In [ ]:
df_clean.drop(columns=['station']).to_csv('data/src/knmi-processed.csv', index=False)